<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkCV0101ENCoursera872-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


<h1>Logistic Regression With Mini-Batch Gradient Descent </h1> 


<h2>Objective</h2>

<ul>
    <li>Represent your data as a Dataset object</li>
    <li>Create a Logistic Regression Model using PyTorch</li>
    <li>Set a Criterion to calculate Loss</li>
    <li>Create a Data Loader and set the Batch Size</li>
    <li>Create an Optimizer to update Model Parameters and set Learning Rate</li>
    <li>Train a Model</li>
</ul> 


<h2>Table of Contents</h2>
<p>In this lab, you will learn how to train a PyTorch Logistic Regression model using Mini-Batch Gradient Descent.</p>

<ul>
    <li>Load Data</li>
    <li>Create the Model and Total Loss Function (Cost)</li>
    <li>Setting the Batch Size using a Data Loader</li>
    <li>Setting the Learning Rate</li>
    <li>Train the Model via Mini-Batch Gradient Descent</li>
    <li>Question</li>
</ul>


<hr>


<h2>Preparation</h2>


In [ ]:
!pip3 install torch torchvision torchaudio

In [1]:
# Import the libraries we need for this lab

# Allows us to use arrays to manipulate and store data
import numpy as np
# Used to graph data and loss curves
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
# PyTorch Library
import torch
# Used to help create the dataset and perform mini-batch
from torch.utils.data import Dataset, DataLoader
# PyTorch Neural Network
import torch.nn as nn




Equation of linear classifier : <code>Z = wx+b </code>. x is input feature vector.The linear classifier assigns a class label to an input based on the sign of the predicted output. If Z is positive, the input is classified into one class, and if Z is negative, it is classified into another class. 


------------------

for x=2, plot the  decision plane Z = wx+b . The line  where the decision plane intersects the x-y plane is the decision boundary. anything on the left is one class and on the right is another class.

----------

Well it is not guaranted that decision boundary is a linear. So we need a threshold function to what extent our result is guaranted.Threshold function is  sigma(z) = 1/(1+ e^-z) If the value of sigmoid function is greater than 0.5 ,is one class and if less then another class.

-------




We can use loss function and cost function to determine the best parameters. A loss function tells you how good our predictions is. Whenever it predicts wrong class ,loss is 1 and when correct loss is 0 . cost is the summation of all the losses. So cost is a function of learnable parameters.

-------------

The cross-entropy loss  calculated as follows:

L(y, ŷ) = - (y * log(ŷ) + (1 - y) * log(1 - ŷ))

where:

    L(y, ŷ) is the cross-entropy loss,
    y is the true label (0 or 1),
    ŷ is the predicted probability of the positive class (between 0 and 1).

The loss function penalizes the model more when it makes incorrect predictions with high confidence. If the true label (y) is 1, the first term in the loss function penalizes the model when the predicted probability (ŷ) is low. If the true label is 0, the second term penalizes the model when the predicted probability is high.

During training, the goal is to minimize the average cross-entropy loss over the entire training dataset. This is typically done using optimization algorithms such as gradient descent, which adjust the model's weights and biases to find the optimal values that minimize the loss.

In [34]:
# Create plot_error_surfaces class ,child of object class .In this lab we use logistic regression of input feature 1 so w is a  value and b is also a value.

class plot_error_surfaces(object):
    
    # Construstor
    def __init__(self, w_range, b_range, X, Y, n_samples = 30, go = True):
        W = np.linspace(-w_range, w_range, n_samples)     #n_samples equally spaced points in that w_range
        B = np.linspace(-b_range, b_range, n_samples)
        
        w, b = np.meshgrid(W, B)    # possible combinations of W and B. each are 30X30 array.
        #print(w) 
        #print(b)
        
        Z = np.zeros((30, 30))     # to store average loss for each combinations of w and b.
        count1 = 0
        #X is the input vector and y is the output vector .
        self.y = Y.numpy()  
        self.x = X.numpy()
        for w1, b1 in zip(w, b):     # w1 and b1 are array of size 30 each .w1 is W itself while b1 is the 30X1  array of one value of B.
             
            count2 = 0
            for w2, b2 in zip(w1, b1):
            
                 
                yhat= 1 / (1 + np.exp(-1*(w2*self.x+b2)))   # self.x is a vector .yhat is sigmoid of z.
                
                #print(w2*self.x+b2)              #it is also a array.
                #print(yhat)                        #yhat is also a array of yhat value of corresponding inputs.
                
                Z[count1,count2]=-1*np.mean(self.y*np.log(yhat+1e-16) +(1-self.y)*np.log(1-yhat+1e-16))    # cross entropy loss for that pair of w2 and b2.
                count2 += 1   
            count1 += 1
        self.Z = Z
        self.w = w
        self.b = b
        self.W = []
        self.B = []
        self.LOSS = []
        self.n = 0
        if go == True:
            plt.figure()
            plt.figure(figsize=(7.5, 5))
            plt.axes(projection='3d').plot_surface(self.w, self.b, self.Z, rstride=1, cstride=1, cmap='viridis', edgecolor='none')
            plt.title('Loss Surface')
            plt.xlabel('w')
            plt.ylabel('b')
            plt.show()
            plt.figure()
            plt.title('Loss Surface Contour')
            plt.xlabel('w')
            plt.ylabel('b')
            plt.contour(self.w, self.b, self.Z)
            plt.show()
            
     # Setter
     #This function, set_para_loss, is used to store the values of model parameters (W and B) and the corresponding loss (LOSS) during training.
    def set_para_loss(self, model, loss):
        self.n = self.n + 1     # no of iterations 
        #print(self.n)
        self.W.append(list(model.parameters())[0].item())  # corresponding w and b value.
        self.B.append(list(model.parameters())[1].item())
        #print(list(model.parameters())[0].item())
        #print(list(model.parameters())[1].item())
        self.LOSS.append(loss)
    
    # Plot diagram
    def final_plot(self): 
        ax = plt.axes(projection='3d')
        ax.plot_wireframe(self.w, self.b, self.Z)
        ax.scatter(self.W, self.B, self.LOSS, c='r', marker='x', s=200, alpha=1)
        plt.figure()
        plt.contour(self.w, self.b, self.Z)
        plt.scatter(self.W, self.B, c='r', marker='x')
        plt.xlabel('w')
        plt.ylabel('b')
        plt.show()
        
    # Plot diagram
    def plot_ps(self):
        plt.subplot(121)
        plt.ylim
        plt.plot(self.x[self.y==0], self.y[self.y==0], 'ro', label="training points")
        plt.plot(self.x[self.y==1], self.y[self.y==1]-1, 'o', label="training points")
        plt.plot(self.x, self.W[-1] * self.x + self.B[-1], label="estimated line")
        plt.xlabel('x')
        plt.ylabel('y')
        plt.ylim((-0.1, 2))
        plt.title('Data Space Iteration:' + str(self.n))
        plt.show()
        plt.subplot(122)
        plt.contour(self.w, self.b, self.Z)
        plt.scatter(self.W, self.B, c='r', marker='x')
        plt.title('Loss Surface Contour Iteration' + str(self.n))
        plt.xlabel('w')
        plt.ylabel('b')
        
# Plot the diagram

def PlotStuff(X, Y, model, epoch, leg=True):
    
    plt.plot(X.numpy(), model(X).detach().numpy(), label=('epoch ' + str(epoch)))
    plt.plot(X.numpy(), Y.numpy(), 'r')
    if leg == True:
        plt.legend()
    else:
        pass

Set the random seed:


In [3]:
# Setting the seed will allow us to control randomness and give us reproducibility
torch.manual_seed(0)

<!--Empty Space for separating topics-->


<h2 id="Makeup_Data">Load Data</h2>


The Dataset class represents a dataset. Your custom dataset should inherit Dataset which we imported above and override the following methods:

<p><code>__len__</code> so that len(dataset) returns the size of the dataset.</p>
<p><code>__getitem__</code> to support the indexing such that dataset[i] can be used to get ith sample</p>

Below we will create a sample dataset


In [4]:
# Create the custom Data class which inherits Dataset as parent class.
class Data(Dataset):
    
    # Constructor
    def __init__(self):
        # Create X values from -1 to 1 with step .1
        self.x = torch.arange(-1, 1, 0.1).view(-1, 1)  # 20X1 tensor with 1(arguement as 1 given) column.
        # Create Y values all set to 0
        self.y = torch.zeros(self.x.shape[0], 1)
        # Set the X values above 0.2 to 1
        self.y[self.x[:, 0] > 0.2] = 1    #This line sets the values of self.y to 1 where the corresponding values in self.x (first column) are greater than 0.2.
        # Set the .len attribute because we need to override the __len__ method
        self.len = self.x.shape[0]
        #print(self.x.shape[1])
    
    # Getter that returns the data at the given index
    def __getitem__(self, index):      
        return self.x[index], self.y[index]
    
    # Get length of the dataset
    def __len__(self):
        return self.len

Make <code>Data</code> object


In [7]:
# data_set is a Data object
data_set = Data()


We can see the X values of the dataset


In [ ]:
data_set.x

We can see the Y values of the dataset which correspond to the class of the X value


In [ ]:
data_set.y

We can get the length of the dataset


In [18]:
len(data_set)

20

We can get the label $y$ as well as the $x$ for the first sample 


In [19]:
x,y = data_set[0]  #data_set is 20X2 tensor.
print("x = {},  y = {}".format(x,y))

x = tensor([-1.]),  y = tensor([0.])


We can get the label $y$ as well as the $x$ for the second sample:


In [ ]:
x,y = data_set[1]
print("x = {},  y = {}".format(x,y))

 We can see we can separate the one-dimensional dataset into two classes:


In [ ]:
plt.plot(data_set.x[data_set.y==0], data_set.y[data_set.y==0], 'ro', label="y=0")
plt.plot(data_set.x[data_set.y==1], data_set.y[data_set.y==1]-1, 'o', label="y=1")
plt.xlabel('x')
plt.ylabel('y')
plt.legend()          

<!--Empty Space for separating topics-->


<h2 id="Model_Cost">Create the Model and Total Loss Function (Cost)</h2>


We will create a custom class that defines the architecture of Logistic Regression using PyTorch. Logistic Regression has a single layer where the input is the number of features an X value of the dataset has (dimension of X) and there is a single output. The output of the layer is put into a sigmoid function which is a function between 0 and 1. The larger the output of the layer the closer it is to 1 and the smaller the output is the closer it is to 0. The sigmoid function will allow us to turn this output into a classification problem. If the output value is closer to 1 it is one class if it is closer to 0 it is in another.


In [12]:
# Create logistic_regression class using PyTorch that inherits nn.Module which is the base class for all neural networks
class logistic_regression(nn.Module):
    
    # Constructor
    def __init__(self, n_inputs):
        super(logistic_regression, self).__init__()      #In the constructor method (__init__), the class is initialized. It takes n_inputs as a parameter, which represents the number of input features for the logistic regression model.
        #The super() function is called to invoke the constructor of the parent class (nn.Module) and initialize the object.
        # Single layer of Logistic Regression with number of inputs being n_inputs and there being 1 output 
        self.linear = nn.Linear(n_inputs, 1)   #This layer will be responsible for the linear mapping of inputs to the output.
        #print(self.linear)
        
    # Prediction
    def forward(self, x):  
        yhat = torch.sigmoid(self.linear(x))   # It takes an input tensor x and passes it through the linear layer (self.linear). The output of the linear layer(z value ,whether it is positive or negative) is then passed through the sigmoid activation function using torch.sigmoid(). Finally, the result is returned as the predicted output yhat.
        return yhat

In [14]:
# We defined the logistic regression class above . Create a logistic  regression  model with no of input features 1.

model = logistic_regression(1)

We can make a prediction sigma $\sigma$ this uses the forward function defined above


In [ ]:
x = torch.tensor([-1.0])
#x2 = torch.tensor([5.0])

#This line calls the forward method of the model object, passing the input tensor x. The forward method performs the forward pass computation of the model, which includes applying the linear transformation and the sigmoid activation function to the input. The output of the forward method is assigned to the variable sigma, which represents the predicted output.
sigma = model(x)        
sigma

Create a <code>plot_error_surfaces</code> object to visualize the data space and the learnable parameters space during training:

We can see on the Loss Surface graph, the loss value varying across w and b values with yellow being high loss and dark blue being low loss which is what we want

On the Loss Surface Contour graph we can see a top-down view of the Loss Surface graph


In [ ]:
# Create the plot_error_surfaces object

# 15 is the range of w
# 13 is the range of b
# data_set[:][0] are all the X values
# data_set[:][1] are all the Y values
#it plots the 3D surface of loss or cost which is a function of  w and b and stores the average loss for each combination of w and b.
get_surface = plot_error_surfaces(15, 13, data_set[:][0], data_set[:][1])

We define a criterion using Binary Cross Entropy Loss. This will measure the difference/loss between the prediction and actual value.


In [21]:
criterion = nn.BCELoss()    #  creates an instance of the Binary Cross Entropy Loss (BCELoss) in PyTorch. The nn.BCELoss() function is part of the torch.nn module.

We have our samples:


In [22]:
x, y = data_set[0]
print("x = {},  y = {}".format(x,y))

x = tensor([-1.]),  y = tensor([0.])


We can make a prediction using the model:


In [ ]:
sigma = model(x)
    # probablity that the input belongs to that class 1 or not.
loss = criterion(sigma, y)
loss

## Setting the Batch Size using a Data Loader


In [26]:
batch_size=10

<code> trainloader</code> = DataLoader(dataset=data_set, batch_size=10): Creates a DataLoader object called trainloader.It takes the data_set as the dataset argument and sets the batch_size to 10(no of samples taken for each iteration). The DataLoader is responsible for loading the data from the dataset in batches for efficient processing.
One epoch is the complete run on the dataset. 

In [27]:
trainloader = DataLoader(dataset = data_set, batch_size = 10)  #it will output a batch of data 

In [28]:
dataset_iter = iter(trainloader)
#dataset_iter = iter(trainloader): Creates an iterator dataset_iter from the trainloader. It allows us to iterate over the batches of data in the trainloader dataset.


In [29]:
X,y=next(dataset_iter )
#X, y = next(dataset_iter): Retrieves the next batch of data from the iterator using the next() function. It returns a tuple (X, y) containing the input features (X) and their corresponding labels (y) for the batch.

We can see here that 10 values the same as our batch size


In [ ]:
X

In [ ]:
y

## Setting the Learning Rate


We can set the learning rate by setting it as a parameter in the optimizer along with the parameters of the logistic regression model we are training. The job of the optimizer, torch.optim.SGD, is to use the loss generated by the criterion to update the model parameters according to the learning rate. SGD stands for Stochastic Gradient Descent which typically means that the batch size is set to 1, but the data loader we set up above has turned this into Mini-Batch Gradient Descent.


In [32]:
learning_rate = 0.1

In [62]:
#torch.optim.SGD(...): Creates an optimizer object of the stochastic gradient descent (SGD) algorithm. It takes the parameters of the model and the learning rate (lr) as arguments. The optimizer will be responsible for updating the model's parameters based on the computed gradients during the training process.
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

<!--Empty Space for separating topics-->


<h2 id="BGD">Train the Model via Mini-Batch Gradient Descent</h2>


We are going to train the model using various Batch Sizes and Learning Rates.


### Mini-Batch Gradient Descent


In this case, we will set the batch size of the data loader to 5 and the number of epochs to 250.

number of epochs: number of times the entire training dataset is passed.


First, we must recreate the get_surface object again so that for each example we get a Loss Surface for that model only.


In [ ]:
get_surface = plot_error_surfaces(15, 13, data_set[:][0], data_set[:][1], 30)

#### Train the Model


In [ ]:
# First we create an instance of the model we want to train
model = logistic_regression(1)
# We create a criterion which will measure loss  using actual y and yhat.
criterion = nn.BCELoss()
# We create a data loader with the dataset and specified batch size of 5
trainloader = DataLoader(dataset = data_set, batch_size = 5)
# We create an optimizer with the model parameters and learning rate
optimizer = torch.optim.SGD(model.parameters(), lr = .01)
# Then we set the number of epochs which is the total number of times we will train on the entire training dataset
epochs=500
# This will store the loss over iterations so we can plot it at the end
loss_values = []  #stores loss for each batch.

# Loop will execute for number of epochs
for epoch in range(epochs):
    # For each batch in the training data
    for x, y in trainloader:
        # Make our predictions from the X values
        yhat = model(x)   # yhat values based on x values.
        # Measure the loss between our prediction and actual Y values
        loss = criterion(yhat, y)   # loss corresponding to this batch
        # Resets the calculated gradient value, this must be done each time as it accumulates if we do not reset
        optimizer.zero_grad()
        # Calculates the gradient value with respect to each weight and bias
        loss.backward()
        # Updates the weight and bias according to calculated gradient value
        optimizer.step()
        # Set the parameters for the loss surface contour graphs
        get_surface.set_para_loss(model, loss.tolist())     #It collects the model's weight and bias values (model.parameters()) and the loss value (loss.tolist()) for visualization purposes.
        # Saves the loss of the iteration
        loss_values.append(loss)
    # Want to print the Data Space for the current iteration every 20 epochs
    if epoch % 20 == 0:
        get_surface.plot_ps()

We can see the final values of the weight and bias. This weight and bias correspond to the orange line in the Data Space graph and the final spot of the X in the Loss Surface Contour graph.


In [77]:
w = model.state_dict()['linear.weight'].data[0]
b = model.state_dict()['linear.bias'].data[0]
print("w = ", w, "b = ", b)

w =  tensor([2.7497]) b =  tensor(-0.7446)


Now we can get the accuracy of the training data. We trained the model , now lets check the accuracy.


In [78]:
# Getting the predictions
yhat = model(data_set.x)
# Rounding the prediction to the nearedt integer 0 or 1 representing the classes
yhat = torch.round(yhat)
# Counter to keep track of correct predictions
correct = 0
# Goes through each prediction and actual y value
for prediction, actual in zip(yhat, data_set.y):
    # Compares if the prediction and actualy y value are the same
    if (prediction == actual):
        # Adds to counter if prediction is correct
        correct+=1
# Outputs the accuracy by dividing the correct predictions by the length of the dataset
print("Accuracy: ", correct/len(data_set)*100, "%")

Accuracy:  100.0 %


Finally, we plot the Cost vs Iteration graph, although it is erratic it is downward sloping.


In [ ]:
plt.plot(loss_values)
plt.xlabel("Iteration")
plt.ylabel("Cost")

### Stochastic Gradient Descent


In this case, we will set the batch size of the data loder to 1 so that the gradient descent will be performed for each example this is referred to as Stochastic Gradient Descent. The number of epochs is set to 100.

Notice that in this example the batch size is decreased from 5 to 1 so there would be more iterations. Due to this, we can reduce the number of iterations by decreasing the number of epochs. Due to the reduced batch size, we are optimizing more frequently so we don't need as many epochs.

First, we must recreate the `get_surface` object again so that for each example we get a Loss Surface for that model only.


In [ ]:
get_surface = plot_error_surfaces(15, 13, data_set[:][0], data_set[:][1], 30)

#### Train the Model


In [ ]:
# First we create an instance of the model we want to train
model = logistic_regression(1)
# We create a criterion which will measure loss
criterion = nn.BCELoss()
# We create a data loader with the dataset and specified batch size of 1
trainloader = DataLoader(dataset = data_set, batch_size = 1)
# We create an optimizer with the model parameters and learning rate
optimizer = torch.optim.SGD(model.parameters(), lr = .01)
# Then we set the number of epochs which is the total number of times we will train on the entire training dataset
epochs=100
# This will store the loss over iterations so we can plot it at the end
loss_values = []

# Loop will execute for number of epochs
for epoch in range(epochs):
    # For each batch in the training data
    for x, y in trainloader:
        # Make our predictions from the X values
        yhat = model(x)
        # Measure the loss between our prediction and actual Y values
        loss = criterion(yhat, y)
        # Resets the calculated gradient value, this must be done each time as it accumulates if we do not reset
        optimizer.zero_grad()
        # Calculates the gradient value with respect to each weight and bias
        loss.backward()
        # Updates the weight and bias according to calculated gradient value
        optimizer.step()
        # Set the parameters for the loss surface contour graphs
        get_surface.set_para_loss(model, loss.tolist())
        # Saves the loss of the iteration
        loss_values.append(loss)
    # Want to print the Data Space for the current iteration every 20 epochs
    if epoch % 20 == 0:
        get_surface.plot_ps()

We can see the final values of the weight and bias. This weight and bias correspond to the orange line in the Data Space graph and the final spot of the X in the Loss Surface Contour graph.


In [82]:
w = model.state_dict()['linear.weight'].data[0]
b = model.state_dict()['linear.bias'].data[0]
print("w = ", w, "b = ", b)

w =  tensor([2.4882]) b =  tensor(-0.7023)


Now we can get the accuracy of the training data


In [83]:
# Getting the predictions
yhat = model(data_set.x)
# Rounding the prediction to the nearedt integer 0 or 1 representing the classes
yhat = torch.round(yhat)
# Counter to keep track of correct predictions
correct = 0
# Goes through each prediction and actual y value
for prediction, actual in zip(yhat, data_set.y):
    # Compares if the prediction and actualy y value are the same
    if (prediction == actual):
        # Adds to counter if prediction is correct
        correct+=1
# Outputs the accuracy by dividing the correct predictions by the length of the dataset
print("Accuracy: ", correct/len(data_set)*100, "%")

Accuracy:  100.0 %


Finally, we plot the Cost vs Iteration graph, although it is erratic it is downward sloping.


In [ ]:
plt.plot(loss_values)
plt.xlabel("Iteration")
plt.ylabel("Cost")

### High Learning Rate


In this case, we will set the batch size of the data loder to 1 so that the gradient descent will be performed for each example this is referred to as Stochastic Gradient Descent. This time the learning rate will be set to .1 to represent a high learning rate and we will observe what will happen when we try to train.

First, we must recreate the `get_surface` object again so that for each example we get a Loss Surface for that model only.


In [ ]:
get_surface = plot_error_surfaces(15, 13, data_set[:][0], data_set[:][1], 30)

#### Train the Model


In [ ]:
# First we create an instance of the model we want to train
model = logistic_regression(1)
# We create a criterion that will measure loss
criterion = nn.BCELoss()
# We create a data loader with the dataset and specified batch size of 1
trainloader = DataLoader(dataset = data_set, batch_size = 1)
# We create an optimizer with the model parameters and learning rate
optimizer = torch.optim.SGD(model.parameters(), lr = 1)
# Then we set the number of epochs which is the total number of times we will train on the entire training dataset
epochs=100
# This will store the loss over iterations so we can plot it at the end
loss_values = []

# Loop will execute for number of epochs
for epoch in range(epochs):
    # For each batch in the training data
    for x, y in trainloader:
        # Make our predictions from the X values
        yhat = model(x)
        # Measure the loss between our prediction and actual Y values
        loss = criterion(yhat, y)
        # Resets the calculated gradient value, this must be done each time as it accumulates if we do not reset
        optimizer.zero_grad()
        # Calculates the gradient value with respect to each weight and bias
        loss.backward()
        # Updates the weight and bias according to calculated gradient value
        optimizer.step()
        # Set the parameters for the loss surface contour graphs
        get_surface.set_para_loss(model, loss.tolist())
        # Saves the loss of the iteration
        loss_values.append(loss)
    # Want to print the Data Space for the current iteration every 20 epochs
    if epoch % 20 == 0:
        get_surface.plot_ps()

Notice in this example the due to the high learning rate the Loss Surface Contour graph has increased movement over the previous example and also moves in multiple directions due to the minimum being overshot.


We can see the final values of the weight and bias. This weight and bias correspond to the orange line in the Data Space graph and the final spot of the X in the Loss Surface Contour graph.


In [ ]:
w = model.state_dict()['linear.weight'].data[0]
b = model.state_dict()['linear.bias'].data[0]
print("w = ", w, "b = ", b)

Now we can get the accuracy of the training data


In [ ]:
# Getting the predictions
yhat = model(data_set.x)
# Rounding the prediction to the nearedt integer 0 or 1 representing the classes
yhat = torch.round(yhat)
# Counter to keep track of correct predictions
correct = 0
# Goes through each prediction and actual y value
for prediction, actual in zip(yhat, data_set.y):
    # Compares if the prediction and actualy y value are the same
    if (prediction == actual):
        # Adds to counter if prediction is correct
        correct+=1
# Outputs the accuracy by dividing the correct predictions by the length of the dataset
print("Accuracy: ", correct/len(data_set)*100, "%")

Finally, we plot the Cost vs Iteration graph, although it is erratic it is downward sloping.


In [ ]:
plt.plot(loss_values)
plt.xlabel("Iteration")
plt.ylabel("Cost")

## Question


Using the following code train the model using a `learning rate` of `.01`, `120 epochs`, and `batch_size` of `1`.


In [ ]:
get_surface = plot_error_surfaces(15, 13, data_set[:][0], data_set[:][1], 30)

#### Train the Model


In [ ]:
# First we create an instance of the model we want to train
model = logistic_regression(1)
# We create a criterion which will measure loss
criterion = nn.BCELoss()
# We create a data loader with the dataset and specified batch size of 1
trainloader = DataLoader(dataset = data_set, batch_size = "SET_BATCH_SIZE")
# We create an optimizer with the model parameters and learning rate
optimizer = torch.optim.SGD(model.parameters(), lr = "SET_LEARNING_RATE")
# Then we set the number of epochs which is the total number of times we will train on the entire training dataset
epochs= "SET_NUMBER_OF_EPOCHS"
# This will store the loss over iterations so we can plot it at the end
loss_values = []

# Loop will execute for number of epochs
for epoch in range(epochs):
    # For each batch in the training data
    for x, y in trainloader:
        # Make our predictions from the X values
        yhat = model(x)
        # Measure the loss between our prediction and actual Y values
        loss = criterion(yhat, y)
        # Resets the calculated gradient value, this must be done each time as it accumulates if we do not reset
        optimizer.zero_grad()
        # Calculates the gradient value with respect to each weight and bias
        loss.backward()
        # Updates the weight and bias according to calculated gradient value
        optimizer.step()
        # Set the parameters for the loss surface contour graphs
        get_surface.set_para_loss(model, loss.tolist())
        # Saves the loss of the iteration
        loss_values.append(loss)
    # Want to print the Data Space for the current iteration every 20 epochs
    if epoch % 20 == 0:
        get_surface.plot_ps()

<details><summary>Click here for the solution</summary>
<code>  
# First we create an instance of the model we want to train
model = logistic_regression(1)
# We create a criterion which will measure loss
criterion = nn.BCELoss()
# We create a data loader with the dataset and specified batch size of 1
trainloader = DataLoader(dataset = data_set, batch_size = 1)
# We create an optimizer with the model parameters and learning rate
optimizer = torch.optim.SGD(model.parameters(), lr = .01)
# Then we set the number of epochs which is the total number of times we will train on the entire training dataset
epochs= 120
# This will store the loss over iterations so we can plot it at the end
loss_values = []
# Loop will execute for number of epochs
for epoch in range(epochs):
    # For each batch in the training data
    for x, y in trainloader:
        # Make our predictions from the X values
        yhat = model(x)
        # Measure the loss between our prediction and actual Y values
        loss = criterion(yhat, y)
        # Resets the calculated gradient value, this must be done each time as it accumulates if we do not reset
        optimizer.zero_grad()
        # Calculates the gradient value with respect to each weight and bias
        loss.backward()
        # Updates the weight and bias according to calculated gradient value
        optimizer.step()
        # Set the parameters for the loss surface contour graphs
        get_surface.set_para_loss(model, loss.tolist())
        # Saves the loss of the iteration
        loss_values.append(loss)
    # Want to print the Data Space for the current iteration every 20 epochs
    if epoch % 20 == 0:
        get_surface.plot_ps()
</code>
</details>


We can see the final values of the weight and bias. This weight and bias correspond to the orange line in the Data Space graph and the final spot of the X in the Loss Surface Contour graph.


In [ ]:
w = model.state_dict()['linear.weight'].data[0]
b = model.state_dict()['linear.bias'].data[0]
print("w = ", w, "b = ", b)

Now we can get the accuracy of the training data


In [ ]:
# Getting the predictions
yhat = model(data_set.x)
# Rounding the prediction to the nearedt integer 0 or 1 representing the classes
yhat = torch.round(yhat)
# Counter to keep track of correct predictions
correct = 0
# Goes through each prediction and actual y value
for prediction, actual in zip(yhat, data_set.y):
    # Compares if the prediction and actualy y value are the same
    if (prediction == actual):
        # Adds to counter if prediction is correct
        correct+=1
# Outputs the accuracy by dividing the correct predictions by the length of the dataset
print("Accuracy: ", correct/len(data_set)*100, "%")

Finally, we plot the Cost vs Iteration graph, although it is erratic it is downward sloping.


In [ ]:
plt.plot(loss_values)
plt.xlabel("Iteration")
plt.ylabel("Cost")

<!--Empty Space for separating topics-->


<hr>
